## Prototype Notebook

1.31.24

In [1]:
import os
import pandas as pd
import json

results_fp = '../../../datasets/tmp/cake/'
model_names = ['gpt-3.5-turbo', 'gpt-4', 'llama_13b_chat']

In [2]:
!ls {results_fp}

input-common-sense-1.md
input-common-sense-2.md
output-common-sense-1-gpt-3.5-turbo-3749.json
output-common-sense-2-gpt-3.5-turbo-3749.json
readme.md


### Collation + Parsing section

In [3]:
def get_json_result_fns(results_fp):
    results = os.listdir(results_fp)
    results = [r for r in results if r.endswith('.json')]
    results = [r for r in results if r.startswith('output')]
    return results

result_fns = get_json_result_fns(results_fp)
result_fns

['output-common-sense-1-gpt-3.5-turbo-3749.json',
 'output-common-sense-2-gpt-3.5-turbo-3749.json']

In [4]:
def question_table(result_fn, results_fp):
    with open(os.path.join(results_fp, result_fn)) as f:
        data = json.load(f)
    return pd.DataFrame(data['questions'])

q_tbl = question_table(result_fns[0], results_fp)
q_tbl.head(3)

,name,meta_data,ground_truth,question,question_usr,question_sys,completion,error,eval_time,name_model,grading
0,Q-1,{},C) The worm,"In the following, answer the multiple choice q...","In the following, answer the multiple choice q...","In the following, answer the multiple choice q...",C) The worm,None,0.993814,gpt-3.5-turbo,"{'grade_bool': True, 'grade_style': 'fuzzy', '..."
1,Q-2,{},woodchuck\ncould chuck\nwood,"In the following, answer the multiple choice q...","In the following, answer the multiple choice q...","In the following, answer the multiple choice q...",woodchuck could chuck wood.,None,1.078918,gpt-3.5-turbo,"{'grade_bool': False, 'grade_style': 'fuzzy', ..."


In [6]:
def sheet_table_info(result_fn, results_fp):
    with open(os.path.join(results_fp, result_fn)) as f:
        data = json.load(f)
    sheet_data = data.get('header')
    return {
        'input_name':   sheet_data.get('sheet_name'), 
        'model_name':   sheet_data.get('model_name'), 
        'run_id':       sheet_data.get('run_id'),
    }

tbl_info = sheet_table_info(result_fns[0], results_fp)
tbl_info

{'input_name': 'CommonSense QA1', 'model_name': None, 'run_id': '3749'}

In [7]:
def built_full_table(result_fp, result_fn):
    q_tbl = question_table(result_fn, results_fp)
    tbl_info = sheet_table_info(result_fn, results_fp)
    for col_name, col_val in tbl_info.items():
        q_tbl[col_name] = col_val
    return q_tbl

In [8]:
def build_data(results_fp):
    result_fns = get_json_result_fns(results_fp)
    tbls = []
    for result_fn in result_fns:
        tbls.append(built_full_table(results_fp, result_fn))
    return pd.concat(tbls)

In [9]:
data = build_data(results_fp)

In [10]:
data.shape

(4, 14)

In [14]:
def expand_object(data: pd.DataFrame, column: str, drop_original=True):
    try:
        expanded = data[column].apply(pd.Series)
        if drop_original:
            data = data.drop([column], axis=1)
        return pd.concat([data, expanded], axis=1)
    except Exception as e:
        print(f'Error expanding {column}: {e}')
        return data